In [7]:
import pandas as pd
import numpy as np
import os

In [40]:
def load_datasets(path: str) -> list[pd.DataFrame]:
    """
    Loads all CSV files from a directory and returns them as a list of DataFrames.

    Args:
        path (str): Directory containing CSV files.

    Returns:
        list[pd.DataFrame]: List of loaded DataFrames.
    """
    pd_list = []
    
    for filename in os.listdir(path):
        full_path = os.path.join(path, filename)
        if filename.lower().endswith(".csv"):
            df = pd.read_csv(full_path)
            pd_list.append(df)

    return pd_list


In [41]:
a, b, c = load_datasets("Dataset")

# 1. Data Exploration and Manipulation

In [45]:
users = pd.read_csv("Dataset/users.csv", parse_dates=["signup_date"])
pings = pd.read_csv("Dataset/pings.csv", parse_dates=["created_at"])
interactions = pd.read_csv("Dataset/interactions (1).csv", parse_dates=["event_timestamp"])

# --- Engagement scoring weights ---
weight_map = {
    "impression": 0.0,
    "view": 0.2,
    "like": 1.0,
    "comment": 1.5,
    "share": 2.0,
    "follow_creator": 2.5
}

inter = interactions.copy()
inter["event_weight"] = inter["event_type"].map(weight_map)

# merge ping duration to compute watch_time_ratio
inter = inter.merge(
    pings[["ping_id", "duration_sec"]],
    on="ping_id",
    how="left"
)

inter["watch_time_ratio"] = (
    inter["watch_time_sec"] / inter["duration_sec"]
).fillna(0).clip(0, 1)

# final per-event interaction score
inter["interaction_score"] = inter["event_weight"] + inter["watch_time_ratio"]

# --- Aggregate engagement per (user, ping) ---
eng = (
    inter.groupby(["user_id", "ping_id"], as_index=False)
         .agg({
             "interaction_score": "sum",
             "watch_time_sec": "sum",
             "duration_sec": "first"
         })
)

eng["watch_time_ratio"] = (
    eng["watch_time_sec"] / eng["duration_sec"]
).fillna(0).clip(0, 1)

eng["engagement_score"] = eng["interaction_score"]



In [53]:
inter.head(5)

,user_id,ping_id,event_type,watch_time_sec,event_timestamp,event_weight,duration_sec,watch_time_ratio,interaction_score
0,u3,p6,view,9,2024-02-03 17:10:58,0.2,22,0.409091,0.609091
1,u3,p35,like,0,2024-02-01 03:31:06,1.0,24,0.000000,1.000000
2,u5,p31,share,0,2024-02-05 00:59:21,2.0,15,0.000000,2.000000
3,u6,p17,view,40,2024-02-09 14:30:51,0.2,46,0.869565,1.069565
4,u20,p28,follow_creator,0,2024-02-07 02:19:52,2.5,54,0.000000,2.500000


In [54]:
eng.head(5)

,user_id,ping_id,interaction_score,watch_time_sec,duration_sec,watch_time_ratio,engagement_score
0,u1,p30,1.500000,0,51,0.000000,1.500000
1,u1,p35,0.908333,17,24,0.708333,0.908333
2,u1,p37,0.000000,0,57,0.000000,0.000000
3,u1,p6,1.200000,22,22,1.000000,1.200000
4,u10,p13,2.500000,0,58,0.000000,2.500000


# 2a. Distribution of watch_time_ratio for view events

In [59]:
view_events = inter[inter["event_type"] == "view"].copy()

watch_ratio_dist = view_events["watch_time_ratio"].describe(
    percentiles=[0.25, 0.5, 0.75, 0.9]
)
print(watch_ratio_dist)
hist_counts, bin_edges = np.histogram(view_events["watch_time_ratio"], bins=10)
pd.DataFrame({"bin_left": bin_edges[:-1],
              "bin_right": bin_edges[1:],
              "count": hist_counts})

count    102.000000
mean       0.600621
std        0.292695
min        0.051724
25%        0.333333
50%        0.625160
75%        0.867860
90%        0.968304
max        1.000000
Name: watch_time_ratio, dtype: float64


,bin_left,bin_right,count
0,0.051724,0.146552,6
1,0.146552,0.241379,8
2,0.241379,0.336207,13
3,0.336207,0.431034,7
4,0.431034,0.525862,8
5,0.525862,0.620690,8
6,0.620690,0.715517,10
7,0.715517,0.810345,7
8,0.810345,0.905172,13
9,0.905172,1.000000,22


# 2b. Top 10 pings by global engagement score

In [63]:
top_pings = (
    eng.groupby("ping_id")["engagement_score"]
       .sum()
       .sort_values(ascending=False)
       .head(10)
       .reset_index()
       .merge(pings, on="ping_id", how="left")
)

top_pings


,ping_id,engagement_score,creator_id,main_hashtag,category,duration_sec,created_at
0,p32,10.081481,u7,pets,comedy,54,2024-01-16
1,p30,9.354902,u14,pets,gaming,51,2024-01-10
2,p31,8.700000,u5,music,comedy,15,2024-02-03
3,p20,8.500000,u14,travel,education,32,2024-02-23
4,p7,8.229630,u8,music,travel,54,2024-02-22
5,p33,7.919048,u16,fitness,dance,42,2024-01-18
6,p1,7.746429,u3,football,education,56,2024-01-15
7,p6,7.636364,u18,comedy,dance,22,2024-02-04
8,p25,7.377778,u11,makeup,beauty,9,2024-02-23
9,p28,7.344444,u6,makeup,travel,54,2024-01-24


# 2c. New vs Existing Users Comparison

Definition:
A new user = signup_date within 7 days before the last event in the dataset.

In [65]:
latest_ts = interactions["event_timestamp"].max()
cutoff_date = latest_ts - pd.Timedelta(days=7)

users["is_new"] = users["signup_date"] >= cutoff_date

# Focusing on events that imply engagement
engaging_events = interactions[
    interactions["event_type"].isin(["view", "like", "comment", "share"])
].copy()

# attaching duration for watch ratio
engaging_events = engaging_events.merge(
    pings[["ping_id", "duration_sec"]],
    on="ping_id",
    how="left"
)

# Exploring the user-level engagement
user_stats = (
    engaging_events.groupby("user_id")
                   .agg({
                       "watch_time_sec": "sum",
                       "duration_sec": "sum",
                       "ping_id": "nunique"
                   })
                   .rename(columns={"ping_id": "n_interacted_pings"})
)

user_stats["avg_watch_time_ratio"] = (
    user_stats["watch_time_sec"] / user_stats["duration_sec"]
).replace([np.inf, np.nan], 0)

# joining the insights with user metadata
merged = users.merge(user_stats, on="user_id", how="left").fillna(0)

comparison = (
    merged.groupby("is_new")
          .agg({
              "avg_watch_time_ratio": "mean",
              "n_interacted_pings": "mean",
              "user_id": "count"
          })
          .rename(columns={"user_id": "num_users"})
)

comparison


,avg_watch_time_ratio,n_interacted_pings,num_users
is_new,,,
False,0.349595,7.705882,17
True,0.423233,6.333333,3


# Task 1 : Data Understanding and Metrics

## Engagement Score Formula
To quantify user engagement for each (user, ping) pair, I combine two signals: explicit actions (like, comment, share, follow_creator) and implicit watch behavior (watch_time_sec / duration_sec). Explicit actions receive higher weights because they demonstrate clearer intent, while the watch-time ratio provides a continuous measure of attention.
For each interaction event, I compute:

##### event_score = event_weight[event_type] + watch_time_sec / duration_sec

##### The final engagement_score for each (user, ping) is the sum of all event scores for that user on that ping.
This creates a simple, interpretable numeric score that reflects both active and passive engagement.

##### Watch-Time Ratio Distribution
The watch_time_ratio distribution for view events shows:
count = 102
mean = 0.6006
median = 0.6252
std = 0.2927
min = 0.0517
75th percentile = 0.8679
90th percentile = 0.9683
max = 1.0

##### Histogram bucket counts (10 bins):
0.05–0.15: 6
0.15–0.24: 8
0.24–0.33: 13
0.33–0.43: 7
0.43–0.53: 8
0.53–0.62: 8
0.62–0.72: 10
0.72–0.81: 7
0.81–0.90: 13
0.90–1.00: 22

##### Interpretation:
Most users watch at least half of the ping, with a strong cluster near full watches. The distribution is right-skewed, meaning many videos are compelling enough to be watched almost entirely. Lower-ratio buckets still appear, indicating that some content is skipped quickly. Overall, watch-time is a strong implicit engagement signal.

Top 10 Pings by Engagement Score
Using the aggregated engagement_score across all users, the top pings represent those with a combination of strong explicit interactions and high watch-time ratios. These pings likely reflect highly engaging or trending content.

New vs Existing Users
I define new users as those with signup_date within 7 days of the latest event_timestamp in the dataset. Comparing both groups:

Existing users:
avg_watch_time_ratio = 0.3496
n_interacted_pings = 7.71
num_users = 17

New users:
avg_watch_time_ratio = 0.4232
n_interacted_pings = 6.33
num_users = 3

Interpretation:
New users watch a larger fraction of each video, suggesting deeper initial exploration. Existing users interact with more pings overall, indicating more established consumption patterns. The small sample of new users means these values may vary with more data.

What These Metrics Reveal

1. Watch-time ratio is generally high, and many videos are watched almost fully.
2. Explicit interactions amplify engagement scores, but watch-time alone already shows strong discriminatory power.
3. New users watch more deeply but interact with fewer pings, consistent with early exploration.
4. Existing users browse more widely but watch less deeply, reflecting familiarity and selective viewing.
5. A long-tail pattern is present: a few pings accumulate most engagement.
